# Explore District Heating and Kinergy Data

This notebook will

- show the data distribution (how many gas, how many district heating sensors)
- compare statistics about gas and district heating data
- show where the sensors are located
- find missing data spans

## Load Data

In [7]:
import polars as pl

from src.energy_forecast.config import RAW_DATA_DIR

dh_hourly = RAW_DATA_DIR / "district_heating_hourly.csv"
dh_daily = RAW_DATA_DIR / "district_heating_daily.csv"
kinergy_daily = RAW_DATA_DIR / "kinergy_daily.csv"
kinergy_hourly = RAW_DATA_DIR / "kinergy_hourly.csv"

In [8]:
col_list = ["id", "date", "diff", "primary_energy", "adresse", "ort", "plz", "source"]

In [9]:
df_dh_daily = pl.read_csv(dh_daily)
df_dh_daily = df_dh_daily.with_columns(pl.col("date").str.to_date(),
                                       pl.lit("kinergy").alias("source"),
                                       pl.concat_str([pl.col("eco_u_id"), pl.col("data_provider_id")]).alias("id")
                                       ).select(col_list)
df_dh_daily

id,date,diff,primary_energy,adresse,ort,plz,source
str,date,f64,str,str,str,i64,str
"""8f7b3862-a50d-44eb-8ac9-de0cf4…",2022-09-30,147.0,"""district heating""","""Kielort 20""","""Norderstedt""",22850,"""kinergy"""
"""8f7b3862-a50d-44eb-8ac9-de0cf4…",2022-10-01,295.0,"""district heating""","""Kielort 20""","""Norderstedt""",22850,"""kinergy"""
"""8f7b3862-a50d-44eb-8ac9-de0cf4…",2022-10-02,253.0,"""district heating""","""Kielort 20""","""Norderstedt""",22850,"""kinergy"""
"""8f7b3862-a50d-44eb-8ac9-de0cf4…",2022-10-03,315.0,"""district heating""","""Kielort 20""","""Norderstedt""",22850,"""kinergy"""
"""8f7b3862-a50d-44eb-8ac9-de0cf4…",2022-10-04,245.0,"""district heating""","""Kielort 20""","""Norderstedt""",22850,"""kinergy"""
…,…,…,…,…,…,…,…
"""fb5cc271-ae15-4f24-b9d5-30782b…",2024-05-10,10.0,"""district heating""","""Moorbekstraße 33""","""Norderstedt""",22846,"""kinergy"""
"""fb5cc271-ae15-4f24-b9d5-30782b…",2024-05-11,15.0,"""district heating""","""Moorbekstraße 33""","""Norderstedt""",22846,"""kinergy"""
"""fb5cc271-ae15-4f24-b9d5-30782b…",2024-05-12,16.0,"""district heating""","""Moorbekstraße 33""","""Norderstedt""",22846,"""kinergy"""


In [10]:
df_kinergy_daily = pl.read_csv(kinergy_daily)
df_kinergy_daily = df_kinergy_daily.with_columns(pl.col("date").str.to_date(),
                                                 pl.lit("kinergy").alias("source")
                                                 ).rename({"hash": "id"}
                                                 ).select(["id", "date", "Val", "diff", "primary_energy", "qmbehfl", "anzlwhg", "adresse", "ort", "plz", "source"])
df_kinergy_daily

id,date,Val,diff,primary_energy,qmbehfl,anzlwhg,adresse,ort,plz,source
str,date,f64,f64,str,f64,i64,str,str,i64,str
"""2 # JMe4""",2021-08-13,1.0732e6,59.89,"""gas""",1201.78,64,"""Mittlere Schulstraße 4""","""Erlangen""",91054,"""kinergy"""
"""2 # JMe4""",2021-08-14,1.0733e6,189.84,"""gas""",1201.78,64,"""Mittlere Schulstraße 4""","""Erlangen""",91054,"""kinergy"""
"""2 # JMe4""",2021-08-15,1.0736e6,352.56,"""gas""",1201.78,64,"""Mittlere Schulstraße 4""","""Erlangen""",91054,"""kinergy"""
"""2 # JMe4""",2021-08-16,1.0738e6,244.08,"""gas""",1201.78,64,"""Mittlere Schulstraße 4""","""Erlangen""",91054,"""kinergy"""
"""2 # JMe4""",2021-08-17,1.0741e6,186.45,"""gas""",1201.78,64,"""Mittlere Schulstraße 4""","""Erlangen""",91054,"""kinergy"""
…,…,…,…,…,…,…,…,…,…,…
"""20 # SSg1-3""",2023-09-17,2.2951e6,214.7,"""gas""",8125.67,0,"""Slomanstieg 1-3""","""Hamburg""",20539,"""kinergy"""
"""20 # SSg1-3""",2023-09-18,2.2954e6,248.6,"""gas""",8125.67,0,"""Slomanstieg 1-3""","""Hamburg""",20539,"""kinergy"""
"""20 # SSg1-3""",2023-09-19,2.2957e6,350.3,"""gas""",8125.67,0,"""Slomanstieg 1-3""","""Hamburg""",20539,"""kinergy"""


In [13]:
df_daily = pl.concat([df_dh_daily, df_kinergy_daily], how="diagonal")
df_daily.sort(by=["id", "date"])

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""0c9ad311-b86f-4371-a695-512ca4…",2022-09-30,364.0,"""district heating""","""Kielortring 14""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""0c9ad311-b86f-4371-a695-512ca4…",2022-10-01,703.0,"""district heating""","""Kielortring 14""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""0c9ad311-b86f-4371-a695-512ca4…",2022-10-02,334.0,"""district heating""","""Kielortring 14""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""0c9ad311-b86f-4371-a695-512ca4…",2022-10-03,891.0,"""district heating""","""Kielortring 14""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""0c9ad311-b86f-4371-a695-512ca4…",2022-10-04,661.0,"""district heating""","""Kielortring 14""","""Norderstedt""",22850,"""kinergy""",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""fb684f25-a63d-4d3e-9277-6d759b…",2024-05-10,50.0,"""district heating""","""Heidehofweg 99c""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""fb684f25-a63d-4d3e-9277-6d759b…",2024-05-11,51.0,"""district heating""","""Heidehofweg 99c""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""fb684f25-a63d-4d3e-9277-6d759b…",2024-05-12,45.0,"""district heating""","""Heidehofweg 99c""","""Norderstedt""",22850,"""kinergy""",null,null,null


In [19]:
print(f"Total number of datapoints (daily): {len(df_daily)}")
print(f"Total number of sensors: {len(df_daily.group_by(pl.col('id')).agg())}")

Total number of datapoints (daily): 56253
Total number of sensors: 103


## Data distribution

In [26]:
print(f"Number of gas sensors: {len(df_daily.filter(pl.col('primary_energy') == 'gas').group_by(pl.col('id')).agg())}")
print(f"Number of datapoints (daily) for gas sensors: {len(df_daily.filter(pl.col('primary_energy') == 'gas'))}")
print(f"Number of district heating sensors: {len(df_daily.filter(pl.col('primary_energy') == 'district heating').group_by(pl.col('id')).agg())}")
print(f"Number of datapoints (daily) for district heating sensors: {len(df_daily.filter(pl.col('primary_energy') == 'district heating'))}")

Number of gas sensors: 9
Number of datapoints (daily) for gas sensors: 5733
Number of district heating sensors: 94
Number of datapoints (daily) for district heating sensors: 50520


Datapoints (daily) per gas sensors:

In [29]:
df_daily.filter(pl.col('primary_energy') == 'gas').group_by(pl.col("id")).agg(pl.len(),
                                                                              pl.col("date").min().alias("min_date"),
                                                                              pl.col("date").max().alias("max_date"))

id,len,min_date,max_date
str,u32,date,date
"""7 # WHg2-8""",783,2021-07-08,2023-09-13
"""3 # JOe11""",92,2023-06-19,2023-09-18
"""4 # JSe21/23""",752,2021-08-13,2023-09-18
"""8 # WUe4""",783,2021-07-08,2023-09-13
"""19 # SKg63""",391,2022-08-31,2023-09-25
"""2 # JMe4""",752,2021-08-13,2023-09-18
"""6 # WGg8a""",856,2021-05-06,2023-09-13
"""20 # SSg1-3""",606,2022-01-24,2023-09-21
"""17 # SFp36""",718,2021-10-04,2023-09-21


In [35]:
print(f"Average length of datapoints (daily) for gas sensors: {df_daily.filter(pl.col('primary_energy') == 'gas').group_by(pl.col('id')).agg(pl.len()).mean()['len'].item()}")

Average length of datapoints (daily) for gas sensors: 637.0


Datapoints (daily) per district heating:

In [36]:
df_daily.filter(pl.col('primary_energy') == 'district heating').group_by(pl.col("id")).agg(pl.len(),
                                                                              pl.col("date").min().alias("min_date"),
                                                                              pl.col("date").max().alias("max_date"))

id,len,min_date,max_date
str,u32,date,date
"""e7ad9b75-bc6c-4891-a8fd-45e393…",537,2022-09-30,2024-05-14
"""1657f5b3-fad0-4685-b56c-d57982…",608,2022-08-23,2024-05-14
"""fb5cc271-ae15-4f24-b9d5-30782b…",600,2022-08-23,2024-05-14
"""c00c8cba-b6de-4c10-89c0-e92312…",593,2022-08-23,2024-05-14
"""c00c8cba-b6de-4c10-89c0-e92312…",592,2022-08-23,2024-05-14
…,…,…,…
"""fb5cc271-ae15-4f24-b9d5-30782b…",605,2022-08-23,2024-05-14
"""10af300b-a270-4e41-928d-e4048b…",535,2022-09-30,2024-05-14
"""8e9b1544-434e-44a7-8049-8f2e4b…",592,2022-08-23,2024-05-14


In [37]:
print(f"Average length of datapoints (daily) for district heating sensors: {df_daily.filter(pl.col('primary_energy') == 'district heating').group_by(pl.col('id')).agg(pl.len()).mean()['len'].item()}")

Average length of datapoints (daily) for district heating sensors: 537.4468085106383


### Summary

- we have way less gas sensor data than district heating data so far (9/94)
- the gas sensors recorded about 100 days longer on average

## Statistics

In [41]:
df_daily.filter(pl.col("primary_energy") == "gas").describe()["statistic", "diff"]

statistic,diff
str,f64
"""count""",5733.0
"""null_count""",0.0
"""mean""",1140.123146
"""std""",1792.365846
"""min""",0.0
"""25%""",155.601
"""50%""",459.91
"""75%""",1126.61
"""max""",18028.02


In [42]:
df_daily.filter(pl.col("primary_energy") == "district heating").describe()["statistic", "diff"]

statistic,diff
str,f64
"""count""",50520.0
"""null_count""",0.0
"""mean""",342.830245
"""std""",7373.526528
"""min""",-1.031452e6
"""25%""",6.0
"""50%""",124.0
"""75%""",500.0
"""max""",386659.0


## Erroneous Values

In [44]:
df_daily.filter((pl.col("primary_energy") == "district heating") & (pl.col("diff") < 0))

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-14,-530377.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-05-09,-772883.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2023-12-13,-632734.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2023-11-08,-1.031452e6,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-14,-233503.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-05-04,-330862.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null


In [49]:
df_daily.filter((pl.col("primary_energy") == "district heating") & (pl.col("id") == "bc098a2e-0cc7-4f01-b6ad-9d647ae9f627ebf76f41mvxg") & (pl.col("date") > pl.date(2023, 4, 9)) &  (pl.col("date") < pl.date(2023, 5, 6))).sort(by="date")

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-10,190.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-11,180.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-12,45.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-13,0.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-14,0.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-24,0.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-25,0.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""bc098a2e-0cc7-4f01-b6ad-9d647a…",2023-04-26,0.0,"""district heating""","""Ulzburger Straße 459 A""","""Norderstedt""",22846,"""kinergy""",null,null,null


In [52]:
df_daily.filter((pl.col("primary_energy") == "district heating") & (pl.col("id") == "8e9b1544-434e-44a7-8049-8f2e4b14a819c5ebff07syra") & (pl.col("date") > pl.date(2023, 4, 9)))

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-10,112.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-11,111.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-12,106.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-13,108.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-14,-233503.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""8e9b1544-434e-44a7-8049-8f2e4b…",2024-05-10,133.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2024-05-11,147.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2024-05-12,160.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null


In [69]:
df_daily.filter((pl.col("primary_energy") == "district heating") & (pl.col("id") == "5e2fd59d-603a-488b-a525-513541039c4a47e757c2y9pw") & (pl.col("date") > pl.date(2023, 1, 1)))  # große Datenlücke märz bis juli 2023

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""5e2fd59d-603a-488b-a525-513541…",2023-01-02,298.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2023-01-03,428.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2023-01-04,198.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2023-01-07,1157.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2023-01-11,1518.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""5e2fd59d-603a-488b-a525-513541…",2024-05-10,257.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2024-05-11,214.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""5e2fd59d-603a-488b-a525-513541…",2024-05-12,209.0,"""district heating""","""Kielortring 51""","""Norderstedt""",22850,"""kinergy""",null,null,null


In [70]:
df_daily.filter((pl.col("primary_energy") == "district heating") & (pl.col("id") == "d5fb4343-04d4-4521-8a4b-feaf772ff376e225f415kyug") & (pl.col("date") > pl.date(2023, 9, 1)))

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""d5fb4343-04d4-4521-8a4b-feaf77…",2023-09-06,2147.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2023-09-07,0.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2023-09-08,0.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2023-09-09,0.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2023-09-10,0.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""d5fb4343-04d4-4521-8a4b-feaf77…",2024-05-10,251.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2024-05-11,242.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null
"""d5fb4343-04d4-4521-8a4b-feaf77…",2024-05-12,251.0,"""district heating""","""Kielort 21""","""Norderstedt""",22850,"""kinergy""",null,null,null


In [73]:
df_daily.filter((pl.col("primary_energy") == "district heating") & (pl.col("id") == "8e9b1544-434e-44a7-8049-8f2e4b14a819f1af853af3qs") & (pl.col("date") > pl.date(2023, 4, 1)))

id,date,diff,primary_energy,adresse,ort,plz,source,Val,qmbehfl,anzlwhg
str,date,f64,str,str,str,i64,str,f64,f64,i64
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-02,260.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-03,269.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-04,275.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-05,311.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2023-04-06,260.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""8e9b1544-434e-44a7-8049-8f2e4b…",2024-05-10,36.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2024-05-11,36.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null
"""8e9b1544-434e-44a7-8049-8f2e4b…",2024-05-12,39.0,"""district heating""","""Ulzburger Straße 461""","""Norderstedt""",22846,"""kinergy""",null,null,null


### Summary

- there are data holes where diff is zero and the end of the disconnection is marked by a negative large value -> ?
- there are large negative values that are probably mistakes in connection without preceding zeros -> take value of day before